In [3]:
from pynini import *
import os

def procesaLinea(linea):
    chars = [chr(i) for i in range(1,91)] + [r"\[", r"\\", r"\]"] + [chr(i) for i in range(94,256)]
    sigma_star = union(*chars).closure()
    
    R = union( string_map([("<html>", "")]), string_map([("</html>", "")]), string_map([("<p>", "")]), string_map([("</p>", "\n\n")]),string_map([("<br>", "\n")]), string_map([("<\br>", "")]),string_map([("<hr>", "---")]), string_map([("<h1>", "# ")]),string_map([("</h1>", "")]), string_map([("</h2>", "")]),string_map([("<h2>", "## ")]), string_map([("<h3>", "### ")]),string_map([("</h3>", "")]),string_map([("<h4>", "#### ")]),string_map([("</h4>", "")]), string_map([("<h5>", "##### ")]),string_map([("<h5>", "")]), string_map([("<h6>", "###### ")]),string_map([("</h6>", "")]),string_map([("<i>", "*")]),string_map([("</i>", "*")]), string_map([("<b>", "**")]),string_map([("</b>", "**")]), string_map([("<ul>", "\n")]),string_map([("</ul>", "\n")]),string_map([("<li>", "- ")]),string_map([("</li>", "\n")]), string_map([("\n", "\t")]), string_map([("href=\"", "(")]), string_map([("\">", ")")]), string_map([("<a ", "\[")]),string_map([("</a>", "\]")]) )

    #Regla de reescritura
    #Ponemos "","" para aplicarlo a todo el texto
    invent = cdrewrite(R, "", "", sigma_star)
    
    cadena_etiquetada = compose(linea, invent).string()
    
    #print(cadena_etiquetada)
    return cadena_etiquetada
    
    
def nth_repl(s, sub, repl, n):
    find = s.find(sub)
    #Si la variable find no es -1, significa que hemos encontrado al menos una ocurrencia para el substring
    i = find != -1
    #Bucle hasta que encontremos el nth o hasta que no encontremos un match
    while find != -1 and i != n:
        # find + 1 significa que empezamos a buscar desde el ultimo match
        find = s.find(sub, find + 1)
        i += 1
    # Si i es igual a n, hemos encontrado el nth match, por lo que realizamos el intercambio
    if i == n:
        return s[:find] + repl + s[find+len(sub):]
    return s    
    

def intercambio(line):
    
    #Comienzo obteniendo los indices de los parantesis de apertura y cierre y del corchete de cierre.
    check = line.find('(')
    check2 = line.find(')')
    check3 = line.find(']')

    #Mediante esta sentencia obtengo el texto que hay entre parentesis
    aux1 = line[check:check2+1]

    #Con esto, obtengo el texto que viene a continuacion del cierre de parentesis hasta el cierre del corchete(incluido)
    aux2 = line[check2+1:check3+1]

    #Intercambio #1
    line2 = line.replace(aux1, aux2)

    #Despues de este intercambio, nos queda la misma expresion 2 veces, por lo que nos interesa hacer el reemplazo de la segunda ocurrencia. 
    #Esto lo conseguimos mediante la funcion nth_repl

    #Intercambio #2
    line3 = nth_repl(line2, aux2, aux1, 2)

    
    return line3

In [4]:
f = open("texto.html", "r")
lineas = f.readlines()

file = open("Cabanilla_Jhon.md", "w")

for i in range(len(lineas)):
    
    if("a href=") not in lineas[i]:
        cadena = procesaLinea(lineas[i])
    
    else:
        cadena_2 = procesaLinea(lineas[i])
        cadena = intercambio(cadena_2)
        
    #Escribimos en el fichero
    file.write(cadena + os.linesep)
        
file.close()